In [ ]:
import warnings
warnings.filterwarnings('ignore')
%reload_ext autoreload
%autoreload 2
from vision_ai_api import APIVisionAI
from IPython.display import Markdown
import pandas as pd
import io
import requests
import json

## API


In [ ]:
def get_objects_table_from_sheets(
    url: str = "https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=1672006844",
):
    request_url = url.replace("edit#gid=", "export?format=csv&gid=")
    response = requests.get(request_url)
    dataframe = pd.read_csv(io.StringIO(response.content.decode("utf-8")), dtype=str)
    dataframe["label"] = dataframe["label"].fillna("null")
    dataframe = dataframe[dataframe["use"] == "1"]
    dataframe = dataframe.drop(columns=["use"])

    return dataframe


df = get_objects_table_from_sheets()
labels_to_put = df.to_dict("records")

In [ ]:
with open("secrets.json") as f:
    s = json.load(f)

vision_ai_api = APIVisionAI(
    username=s.get("username"),
    password=s.get("password"),
)

objects = vision_ai_api._get_all_pages("/objects")
cameras = vision_ai_api._get_all_pages("/cameras", page_size=3000)
prompts = vision_ai_api._get_all_pages("/prompts")

In [ ]:
## GET A OBJECT Parameters
object_slug = "traffic"
object_parameters = vision_ai_api.get_item_from_slug(slug=object_slug, data=objects)

In [ ]:
## GET ALL active cameras

cameras_ativas = vision_ai_api._get_all_pages(
    "/agents/89173394-ee85-4613-8d2b-b0f860c26b0f/cameras"
)
cameras_ativas_ids = [f"/cameras/{d.get('id')}" for d in cameras_ativas]
cameras_agent = vision_ai_api._get_all_pages(cameras_ativas_ids)

In [ ]:
# # ### POST CAMERA TO AGENT

# cameras_to_input = [
#     '222',
#     '99',
#     '793',
#     '100',
#     '794',
#     '793',
#     '1613',
#     '1612',
#     '795',
#     '1594',
#     '10',
#     '2',
#     '1624',
#     '108',
#     '1234',
#     '1231',
#     "1175",
#     "1316",
#     "1189",
#     "276",
#     "33",
#     "1472",
#     "3306",
#     "221",
#     "52",
#     "270",
#     "96",
#     "1615",
#     "1623",
#     "211",
#     "1496",
#     "1497",
#     "1390",
#     "211",
#     "102",
#     "362",
#     "432",
#     "260"
# ]

# agent_id = "89173394-ee85-4613-8d2b-b0f860c26b0f"


# for camera_id in cameras_to_input:
#     camera_id = camera_id.zfill(6)
#     r = vision_ai_api._post(
#         path=f'/agents/{agent_id}/cameras?camera_id={camera_id}'
#     )
#     print(r)

In [ ]:
print(
    '\n### Role \n\n- **Urban Road Image Analyst**\n\n### Objective\n    \n- **Task**: Analyze CCTV images, specifically focusing on elements related to urban road conditions.\n\n### Input\n\n- **Data Provided**: A CCTV image.\n\n### Objects Table \n\n- **Guidance**: Use the table below for object classification, adhering to the specified criteria and identification guides.\n\n| object               | label             | criteria                                                              | identification_guide                                                                                                                                                                                                                                                                                               |\n|:---------------------|:------------------|:----------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|\n| image_condition      | poor              | The image has visual interference affecting clarity.                  | Examine the image for uniform grey or green color or distortions indicating data loss or image corruption.                                                                                                                                                                                                         |\n| image_condition      | clean             | The image is clear and free of visual interferences.                  | Clean image with absence of uniform grey or green color or distortions indicating data loss or image corruption.                                                                                                                                                                                                   |\n| image_description    | null              | Image description and visual elements from the image.                 | Provide a detailed, unbiased description of the image, focusing on visible aspects for analysis. Minimum word count: 500.                                                                                                                                                                                          |\n| road_blockade        | free              | The road is clear, without any obstructions impeding traffic flow.    | Confirm the road is free from obstructions. Normal flow of cars, bicycles, and other vehicles indicates a \'free\' road label. Natural water bodies (canals, rivers, lakes, etc) adjacent to but not obstructing the road also indicate a clear path. Trees alongside or near the road indicates a \'free\' road label |\n| road_blockade        | partially_blocked | Obstructions present that partially impede traffic but allow passage. | Look for indications of partial road coverage by any elements that slow trafic flow. Partial road coverage by water should be considered.                                                                                                                                                                          |\n| road_blockade        | totally_blocked   | The road is completely inaccessible due to obstructions.              | Identify any significant obstructions like extensive water flooding that make the road entirely impassable, offering no alternative paths.                                                                                                                                                                         |\n| traffic_ease_vehicle | easy              | Minimal or no water on the road.                                      | Look for clear, dry, or slightly wet road surfaces and small, manageable puddles that do not interfere with traffic                                                                                                                                                                                                |\n| traffic_ease_vehicle | difficult         | A partial portion of the road is covered with medium water level      | Identify areas where water coverage is extensive and high, causing notable hindrance to vehicle.                                                                                                                                                                                                                   |\n| traffic_ease_vehicle | impossibe         | Complete submergence of the road with high water level                | Identify a scenarios where the road is entirely submerged/flooded, making it completely impassable for vehicles.                                                                                                                                                                                                   |\n| traffic_ease_vehicle | moderate          | Presence of significant, larger puddles                               | Detect larger puddles that could cause minor traffic disruptions but are still navigable for most vehicles.                                                                                                                                                                                                        |\n| water_in_road        | false             | No water present on the road; the road is completely dry.             | Look for dry road surface. A dry road should show no signs of moisture, including the absence of puddles, wet spots, and any kind of reflections commonly associated with water.                                                                                                                                   |\n| water_in_road        | true              | Presence of water covering  the road.                                 | Identify areas where water is visibly covering the road surface or slightly wet. Look for consistent, widespread reflections.                                                                                                                                                                                      |\n| water_level          | flodding          | Water completely covers the road\'s surface.                           | Identify areas where water completely submerges the road. Visible cues include a lack of visible road surface due to water coverage.                                                                                                                                                                               |\n| water_level          | low_indifferent   | No or minimal water on the road\'s surface.                            | Inspect for dry or slightly wet road. Minimal water is indicated by isolated, small or shallow puddles.  Look for areas with shaddows.                                                                                                                                                                             |\n| water_level          | puddle            | Water covers a significant part of the road.                          | Look for areas where water significantly covers the road. with widespread reflections.                                                                                                                                                                                                                             |\n\n### Scenarios examples:\n\n- Example 1: Dry Road with Clear Traffic\n```json\n{\n    "objects": [\n        {\n            "object": "image_condition",\n            "label_explanation": "Image is clear, no distortion or data loss.",\n            "label": "clean"\n        },\n        {\n            "object": "image_description",\n            "label_explanation": "Urban road in daylight with vehicles, clear weather.",\n            "label": "null"\n        },\n        {\n            "object": "water_in_road",\n            "label_explanation": "Road surface is dry, no signs of water.",\n            "label": "false"\n        },\n        {\n            "object": "water_level",\n            "label_explanation": "No water present, road surface completely dry.",\n            "label": "low_indifferent"\n        },\n        {\n            "object": "traffic_ease_vehicle",\n            "label_explanation": "Traffic is flowing smoothly, no hindrance observed.",\n            "label": "easy"\n        },\n        {\n            "object": "road_blockade",\n            "label_explanation": "Road is completely free of obstructions.",\n            "label": "free"\n        }\n    ]\n}\n```\n\n- Example 2: Partially Flooded Road with Moderate Obstructions\n```json\n{\n    "objects": [\n        {\n            "object": "image_condition",\n            "label_explanation": "Slight blurriness in the image, but generally clear.",\n            "label": "poor"\n        },\n        {\n            "object": "image_description",\n            "label_explanation": "Moderate traffic on an urban road with visible puddles.",\n            "label": "null"\n        },\n        {\n            "object": "water_in_road",\n            "label_explanation": "Puddles observed on parts of the road.",\n            "label": "true"\n        },\n        {\n            "object": "water_level",\n            "label_explanation": "Water covers some parts of the road, forming puddles.",\n            "label": "puddle"\n        },\n        {\n            "object": "traffic_ease_vehicle",\n            "label_explanation": "Traffic moving slower due to water on road.",\n            "label": "moderate"\n        },\n        {\n            "object": "road_blockade",\n            "label_explanation": "Partial obstructions due to water, but traffic can pass.",\n            "label": "partially_blocked"\n        }\n    ]\n}\n```\n\n- Example 3: Fully Flooded and Blocked Road\n```json\n{\n    "objects": [\n        {\n            "object": "image_condition",\n            "label_explanation": "High quality, clear image with no issues.",\n            "label": "clean"\n        },\n        {\n            "object": "image_description",\n            "label_explanation": "Road completely submerged in water, no traffic visible.",\n            "label": "null"\n        },\n        {\n            "object": "water_in_road",\n            "label_explanation": "Road is fully covered in water.",\n            "label": "true"\n        },\n        {\n            "object": "water_level",\n            "label_explanation": "Water level high, road completely submerged.",\n            "label": "flooding"\n        },\n        {\n            "object": "traffic_ease_vehicle",\n            "label_explanation": "Traffic is impossible due to severe flooding.",\n            "label": "impossible"\n        },\n        {\n            "object": "road_blockade",\n            "label_explanation": "Road is entirely blocked by flooding, impassable.",\n            "label": "totally_blocked"\n        }\n    ]\n}\n```\n\n### Output\n\n**Output Order**\n\n- **Sequence**: Follow this order in your analysis: \n    1. image_condition: clean or poor\n    2. image_description: allways null\n    3. water_in_road: true or false\n    4. water_level: low_indifferent, puddle or flooding\n    5. traffic_ease_vehicle: easy, moderate, difficult or impossible\n    6. road_blockade: free, partially_blocked or impossible\n\n- **Importance**: Adhering to this sequence ensures logical and coherent analysis, with each step informing the subsequent ones.\n\n\n\n**Example Format** \n\n- Present findings in a structured JSON format, following the provided example.\n        \n```json\n{\n    "objects": [\n        {\n            "object": "<Object from objects>",\n            "label_explanation": "<Visual description of the image given the object context>",\n            "label": "<Selected label from objects>"\n        }\n    ]\n}\n```\n\n- **Requirement**: Each label_explanation should be a 500-word interpretation of the image, demonstrating a deep understanding of the visible elements.\n\n'
)

In [ ]:
### PUT PROMPT


prompt_text = """

### Role 

- **Urban Road Image Analyst**

### Objective
    
- **Task**: Analyze CCTV images from Rio de Janeiro, specifically focusing on elements related to urban road conditions.

### Input

- **Data Provided**: A CCTV image of an urban road.

### Process

1. **Image Condition Assessment**
    - **Action**: Examine the image for uniform color patches or distortions indicating data loss or image corruption.
    - **Decision**: Assign a label to 'image_condition' reflecting the image's quality based on observed uniform colors, distortions, and blurriness.
    
2. **Detailed Image Description**
    - **Action**: Provide a detailed, unbiased description of the image, focusing on visible aspects for analysis. Minimum word count: 500.
    - **Guideline**: Base your description solely on visible elements in the image, avoiding external assumptions or contexts.

3. **Object Analysis and Classification**
    - **Action**: Use the detailed image description to classify each object per the 'Objects Table'.
    - **Requirement**: Support each classification with clear evidence from the image.

4. **Structured Output Presentation**
    - **Format**: Present findings in a structured JSON format, following the provided example.

### Objects Table 

- **Guidance**: Use the table below for object classification, adhering to the specified criteria and identification guides.

{objects_table_md}


### Output

- **Example Format**: 
        
```json
{output_example}
```

- **Requirement**: Each label explanation should be a 300-word interpretation of the image, demonstrating a deep understanding of the visible elements.

**Output Order**

- **Sequence**: Follow this order in your analysis: 
    1. image_condition
    2. image_description
    3. water_in_road
    4. water_level
    5. traffic_ease_vehicle
    6. road_blockade

- **Importance**: Adhering to this sequence ensures logical and coherent analysis, with each step informing the subsequent ones.

"""


prompt_parameters = vision_ai_api._get("/prompts").get("items")[0]
prompt_id = prompt_parameters.get("id")
request_body = {
    "name": prompt_parameters.get("name"),
    "model": prompt_parameters.get("model"),
    "prompt_text": prompt_text,
    "max_output_token": prompt_parameters.get("max_output_token"),
    "temperature": prompt_parameters.get("temperature"),
    "top_k": prompt_parameters.get("top_k"),
    "top_p": prompt_parameters.get("top_p"),
}


r = vision_ai_api._put(path=f"/prompts/{prompt_id}", json_data=request_body)
print(json.dumps(r.json(), indent=2))

In [ ]:
prompt_parameters = vision_ai_api._get("/prompts").get("items")

In [ ]:
prompt_parameters

In [ ]:
### POST PROMPT


prompt_parameters = vision_ai_api._get("/prompts").get("items")[0]
prompt_id = prompt_parameters.get("id")
request_body = {
    "name": "test",
    "model": prompt_parameters.get("model"),
    "prompt_text": "test",
    "max_output_token": prompt_parameters.get("max_output_token"),
    "temperature": prompt_parameters.get("temperature"),
    "top_k": prompt_parameters.get("top_k"),
    "top_p": prompt_parameters.get("top_p"),
}


r = vision_ai_api._post(path=f"/prompts", json_data=request_body)
print(json.dumps(r.json(), indent=2))